In [77]:
import pandas as pd
from numpy.random import randint
#!pip install git+git://github.com/ivannz/trmf
from functools import reduce
import math
from xgboost import XGBRegressor
from sklearn.base import BaseEstimator, clone
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import ParameterGrid
from trmf import TRMFRegressor
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error as mse_score
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.base import clone
import scipy.sparse as ssp

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
#simplefilter(action='ignore', category=DeprecationWarning)

# 7. Use this file to feed into the clusterwise regression experiments


In [78]:
export = 'Export/ICDM19/'
#Train
import_train_daily = 'import/M4/Daily-train.csv'
import_train_weekly = 'import/M4/Weekly-train.csv'
import_train_hourly = 'import/M4/Hourly-train.csv'
import_train_monthly= 'import/M4/Monthly-train.csv'
import_train_quartely = 'import/M4/Quarterly-train.csv'
import_train_yearly = 'import/M4/Yearly-train.csv'
#Test
import_test_daily = 'import/M4/Daily-test.csv'
import_test_weekly = 'import/M4/Weekly-test.csv'
import_test_hourly = 'import/M4/Hourly-test.csv'
import_test_monthly= 'import/M4/Monthly-test.csv'
import_test_quartely = 'import/M4/Quarterly-test.csv'
import_test_yearly = 'import/M4/Yearly-test.csv'

In [79]:
#clusters 
#import_clusters_daily = 'import/M4/Daily-test.csv'
import_clusters_weekly = pd.read_csv('import/M4/Clusters/ensemble_clusterings_m4_weekly.csv')
import_clusters_hourly = pd.read_csv('import/M4/Clusters/ensemble_clusterings_m4_hourly.csv')
import_clusters_monthly= pd.read_csv('import/M4/Clusters/ensemble_clusterings_m4_monthly.csv')
import_clusters_quartely = pd.read_csv('import/M4/Clusters/ensemble_clusterings_m4_quarterly.csv')
import_clusters_yearly = pd.read_csv('import/M4/Clusters/ensemble_clusterings_m4_yearly.csv')

In [80]:
# Train
#train_daily = pd.read_csv(import_train_daily)
train_weekly = pd.read_csv(import_train_weekly)
#train_hourly = pd.read_csv(import_train_hourly)
#train_monthly= pd.read_csv(import_train_monthly)
#train_quartely = pd.read_csv(import_train_quartely)
#train_yearly = pd.read_csv(import_train_yearly)
# Test
#test_daily = pd.read_csv(import_test_daily)
test_weekly = pd.read_csv(import_test_weekly)
#test_hourly = pd.read_csv(import_test_hourly)
#test_monthly= pd.read_csv(import_test_monthly)
#test_quartely = pd.read_csv(import_test_quartely)   
#test_yearly = pd.read_csv(import_test_yearly)

In [81]:
# Cut!
train_weekly = train_weekly[:import_clusters_weekly.shape[0]]
test_weekly = test_weekly[:import_clusters_weekly.shape[0]]

#### Change TS so that the all end at the same time

In [82]:
def dataframe_move_nans(df):
    """
    Moves nans in dataframe to the beginning
    Ex:
    1 2 3 nan
    1 2 nan nan
    Result:
    nan   1 2 3
    nan nan 1 2
    """
    df_user = df.V1
    # Temporarily discard column
    df = df.drop(columns = ['V1'])
    # Move nans to the begininng
    df = df.transpose().apply(
        (lambda row:  np.concatenate(
            [np.full([  df.shape[1] - row.dropna().shape[0]], np.nan),
                        row.dropna().values])) , axis = 0).transpose()
    df.insert(0,'V1',df_user)
    return df
    

In [83]:
%%time
#train_daily = dataframe_move_nans(train_daily).fillna(method ='backfill',axis = 1)
train_weekly = dataframe_move_nans(train_weekly).fillna(method ='backfill',axis = 1)
#train_hourly = dataframe_move_nans(train_hourly).fillna(method ='backfill',axis = 1)
#train_monthly = dataframe_move_nans(train_monthly).fillna(method ='backfill',axis = 1)
#train_quartely = dataframe_move_nans(train_quartely).fillna(method ='backfill',axis = 1)
#train_yearly = dataframe_move_nans(train_yearly).fillna(method ='backfill',axis = 1)


CPU times: user 520 ms, sys: 12 ms, total: 532 ms
Wall time: 534 ms


In [84]:
#Add clusters:
##train_hourly.insert(1,'GMM_Pair',import_clusters_hourly.GMM_Pair_ff0)
#train_hourly.insert(1,'GMM_Voting', import_clusters_hourly.GMM_Voting_ff0)

train_weekly.insert(1,'GMM_Pair', import_clusters_weekly.GMM_Pair_ff0)
train_weekly.insert(1,'GMM_Voting', import_clusters_weekly.GMM_Voting_ff0)


#### Cluster assignment (currently random)

In [85]:
train_weekly.insert(1,'random initialization2',randint(0,max(import_clusters_yearly.GMM_Pair_ff0)-1,
                                                       train_weekly.shape[0]))
train_weekly.insert(1,'random initialization1',randint(0,max(import_clusters_yearly.GMM_Voting_ff0)-4,
                                                       train_weekly.shape[0]))

# COLUMN SHOULD BE IN THE BEGINNNG!
#train_daily.insert(1,'random initialization1',randint(0,5,train_daily.shape[0]))
#train_weekly.insert(1,'random initialization1',randint(0,5,train_weekly.shape[0]))

#train_monthly.insert(1,'random initialization1',randint(0,5,train_monthly.shape[0]))
#train_quartely.insert(1,'random initialization1',randint(0,5,train_quartely.shape[0]))
#train_yearly.insert(1,'random initialization1',randint(0,5,train_yearly.shape[0]))

In [86]:
train_weekly.sample()

,V1,random initialization1,random initialization2,GMM_Voting,GMM_Pair,V2,V3,V4,V5,V6,...,V2589,V2590,V2591,V2592,V2593,V2594,V2595,V2596,V2597,V2598
327,W328,2,3,6,6,3828,3828,3828,3828,3828,...,2549,2207,2060,1885,2039,2022,2332,2013,2382,2735


In [87]:
train_weekly.shape

(359, 2602)

In [88]:
# dataframe is ready!

# 8. Do hard clusterwise regression

In [89]:
base = TRMFRegressor(n_components=2,
                 n_order=2,
                 fit_regression=False,
                 fit_intercept=True,
                 nonnegative_factors=True,
                 n_max_mf_iter=5)


def gridsearch_trmf(X,test_targets):
    grid = ParameterGrid(dict(
    n_components=np.r_[1:2],#17],
    n_order=np.r_[1:2],#17],
    C_Z=np.logspace(-2, 1,num=1),#,num=4),
    C_F=np.logspace(-2, 1,num=1),#,num=4),
    C_phi=np.logspace(-2, 1,num=1),#,num=4),
    eta_Z=np.linspace(0.05, 0.95,num=1)#,num=10),
    ))
    
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = transformer.fit_transform(X)

    
    # run the experiment in parallel
    par_ = Parallel(n_jobs=-1, verbose=0)
    results = par_(delayed(helper)(par, X, test_targets.transpose(),transformer) for par in grid)
    
    keys = ['n_order', 'n_components', 'eta_Z', 'C_phi', 'C_Z', 'C_F']
    data = dict((tuple(par[k] for k in keys), rmse,) for par, rmse in results)
    sr = pd.Series(data, name="rmse").sort_index().rename_axis(keys)
    cube = sr.values.reshape(*[len(grid.param_grid[0][k]) for k in keys])
    stepping = [grid.param_grid[0][k] for k in keys]
    # find the flat index of the smallest value
    flat_index = np.argmin(cube)
    # ... and unravel into into a multidimensional index
    index = np.unravel_index(flat_index, cube.shape)
    # collect the best paramaters from the grid
    best_ = {k: grid.param_grid[0][k][i] for k, i in zip(keys, index)}
    return best_

transformer = StandardScaler(with_mean=True, with_std=True)

def helper(par, train, test,  transformer,base=base):
    basee = TRMFRegressor(n_components=1,
                     n_order=0,
                     fit_regression=False,
                     fit_intercept=True,
                     nonnegative_factors=True,
                     n_max_mf_iter=5)
    
    # clone, set parameters and fit
    trmf = clone(basee).set_params(**par).fit(train)
    
    # predict and return
    pred = transformer.inverse_transform(
        trmf.predict(n_ahead=len(test)))

    return par, mean_squared_error(test, pred)

In [90]:
def smape(y_true,y_pred,training_series= None): 
    weight = (np.abs(y_true) + np.abs(y_pred)) / 2
    output = np.divide(np.abs(y_true - y_pred), weight,
    where=weight > 0,
    out=np.full_like(weight, np.nan))
    return np.mean(np.nan_to_num(output))

In [91]:
def smape_(y_true,y_pred,training_series= None): 
    weight = (np.abs(y_true) + np.abs(y_pred)) / 2
    output = np.divide(np.abs(y_true - y_pred), weight,
    where=weight > 0,
    out=np.full_like(weight, np.nan))
    return np.mean(np.nan_to_num(output))

In [92]:
def pinball_loss(A,F,training_series= None,tau = 0.49):
    return np.mean(np.maximum(F - A, 0) * (1 - tau) +
                 np.maximum(F - A, 0) * tau)

In [93]:
def mape(y_true, y_pred,training_series= None): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [94]:
def MASE(testing_series, prediction_series,training_series):
    n = training_series.shape[0]
    d = np.abs(  np.diff( training_series) ).sum()/(n-1)
    errors = np.abs(testing_series - prediction_series )
    return errors.mean()/d

In [95]:
def clusterwise_regression_sklearn(X,Y,regr,regr_gridsearch_params,
                           cluster_column ='supercluster',
                            horizon = True,
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           recalculate_assignment = 100,
                           score_function = (lambda x,y: math.sqrt(mean_squared_error(x,y))),
                          ):
    """
    Runs clusterwise regression for sklearn functions
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    cluster_column : column with given clusters
    horizon = :  True  - 1-step horizon (1 can be modified), False - n periods ahead
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    gridsearch_limit : # of times to run GridSeach for each cluster (>=1)
    recalculate_assignment : # of assignments after which the clusters will be recalculated
    score_function : smape/ pinball_loss/ mape/(lambda x,y: math.sqrt(mean_squared_error(x,y)))
    regr : if not tmrf, then use sklearn api
    regr_gridsearch_params : if not tmrf, then use sklearn api for gridsearchCV
    ---------
    Return :
    clusters : Dictionary with trained models for each cluster
    old_cluster_for_user : Dictionary with optimal cluster for each user (after moving)
    cluster_parameters_for_gridsearch : Dictionary with optimal parameters for each cluster model
    weights
    """   
    train_targets = X.copy()
    test_targets = Y.copy()  
    n_ahead = test_targets.shape[1]
    clusters = {}
    # save cluster info for clusterwise regression
    cluster_error = {}
    cluster_has_changed = {}
    cluster_data_X = {}
    counter_for_gridsearch = 0
    old_clusters ={}
    cluster_data_true_values = {}
    cluster_predicted = {}
    score_for_user = {}
    cluster_parameters_for_gridsearch = {}
    previous_total_error = math.inf
    limit = 50
    for i in list(range(0,limit+1)): # Some limit on iterations just in case
        # Assign user to clusters
        cluster_for_user = {}
        cluster_for_user  = train_targets[cluster_column].to_dict()
        for cluster_number in train_targets[cluster_column].unique():
            # Save training data for this cluster
            
            cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
            cluster_data_true_values[cluster_number] = test_targets.loc[ cluster_data_X[cluster_number].index]
            cluster_predicted[cluster_number] =pd.DataFrame()
            if (cluster_number not in cluster_has_changed) or (cluster_has_changed[cluster_number]== True):
                    clusters[cluster_number] = clone(regr)
                    cluster_predicted[cluster_number] =pd.DataFrame()
            for slide in list(range(0,test_targets.shape[1])):
                #Train
                transformer = StandardScaler(with_mean=True, with_std=True)
                #print("X_train = ",cluster_data_X[cluster_number].iloc[:,:-1].columns)
                fitted_ = transformer.fit_transform(cluster_data_X[cluster_number].iloc[:,:-1]) # X - all col except last one
                #print("y_train = ",cluster_data_X[cluster_number].iloc[:,-1:].columns)
                #print(cluster_data_X[cluster_number].iloc[:,-1:])
                clusters[cluster_number].fit(fitted_,cluster_data_X[cluster_number].iloc[:,-1:].values) # y - last column
                #Test
                #delete first col
                cluster_data_X[cluster_number] = cluster_data_X[cluster_number].drop(cluster_data_X[cluster_number].columns[0], axis=1)  
                #print("X_test = ",cluster_data_X[cluster_number].columns)
                fitted_ = transformer.fit_transform(cluster_data_X[cluster_number])
                cluster_predicted[cluster_number][slide] =clusters[cluster_number].predict(fitted_).ravel() 
                #add new one at the end (what we've just predicted)
                if (horizon == False):
                    cluster_data_X[cluster_number][slide] = cluster_predicted[cluster_number][slide].values 
                else:
                    cluster_data_X[cluster_number][slide] =(cluster_data_true_values[cluster_number].iloc[:,slide])   # true values
                #print("y_true = ",cluster_data_true_values[cluster_number].columns[slide])
           # return cluster_predicted           
            # Save prediction
            cluster_predicted[cluster_number] = pd.DataFrame(cluster_predicted[cluster_number])

            cluster_predicted[cluster_number].columns  = cluster_data_true_values[cluster_number].columns 
            cluster_predicted[cluster_number].index = cluster_data_true_values[cluster_number].index
            #save cluster score
            cluster_error[cluster_number] =  score_function(cluster_data_true_values[cluster_number].values.ravel(),cluster_predicted[cluster_number].values.ravel())
        print("Error per cluster: ",cluster_error)
        # flatten true values
        all_true_values  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_data_true_values.values())  ]])   )
        # flatten prediction 
        all_predictions  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_predicted.values())  ]])   )
        total_error =  score_function(np.array(all_true_values),np.array(all_predictions))
        print("New error: ",round(total_error,4))
        # If new score is worse - load last cluster and run final prediction.
        if (total_error > previous_total_error):
            if counter_iterations ==3:
               # train_targets[cluster_column] = list(old_cluster_for_user.values())
                print('Error has increased. Backtrack to previous iteration')
                return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch,i-counter_iterations
            counter_iterations+=1
        else:
            counter_iterations = 0 
            previous_total_error = total_error
            old_clusters = clusters.copy()
            old_cluster_for_user = cluster_for_user.copy()
       
        if (i>=limit):# return prev iteration
            return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch,i
        # Calculate score per user
        for cluster_number in train_targets[cluster_column].unique():
            for index, row in cluster_data_true_values[cluster_number].iterrows():
                score_for_user[index] = score_function(row, cluster_predicted[cluster_number].loc[index,:])
        # Assume that we haven't change any cluster
        for x in cluster_has_changed:
            cluster_has_changed[x]=False        
        cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
       # old_clusters = {}
       # old_cluster_for_user = {}
        ii =0
        reassignment_count = 0 
        if (i==0):
            old_clusters = clusters.copy()
        #Take observation x from cluster a, where error_x > error_a
        for cluster_number in train_targets[cluster_column].unique():
            # All ids in this cluster are already in cluster with lowest possible mae
            if (cluster_number == cluster_lowest_mae):
                continue
            for index, row in cluster_data_true_values[cluster_number].iterrows():    
            # User is in current cluster -> we can reassign him
                if (cluster_number  == cluster_for_user[index]):
                    if (score_for_user[index] > cluster_error[cluster_number]):
                        ii+=1#counter
                        # Have to refit changed clusters             
                        cluster_has_changed[cluster_number] = True
                        cluster_has_changed[cluster_lowest_mae] = True
                        # Move observation x to the cluster with the lowest training erro
                        train_targets.loc[index,cluster_column] = cluster_lowest_mae
                        reassignment_count+=1
                        if reassignment_count==recalculate_assignment:
                            reassignment_count=0
                            
                            # RECALCULATION - only best performing cluster (because)- TODO:save snapshot of prev cluster for backtracking
                            cluster_data_X_reassigned =  train_targets.loc[train_targets[cluster_column] == cluster_lowest_mae].drop(columns =columns_wo_TS)
                            cluster_data_true_values_reassigned= test_targets.loc[ cluster_data_X_reassigned.index]
                           
                            clusters_reassignemnt = clone(clusters[cluster_lowest_mae])
                            cluster_predicted_reassigned = pd.DataFrame()
                            for slide in list(range(0,test_targets.shape[1])):
                                #Train
                                transformer = StandardScaler(with_mean=True, with_std=True)
       
                                fitted_ = transformer.fit_transform(cluster_data_X_reassigned.iloc[:,:-1]) # X - all col except last one
                                clusters_reassignemnt.fit(fitted_,cluster_data_X_reassigned.iloc[:,-1:].values) # y - last column
                                #Test
                                #delete first col
                                cluster_data_X_reassigned = cluster_data_X_reassigned.drop(cluster_data_X_reassigned.columns[0], axis=1)  
                                predd = transformer.fit_transform(cluster_data_X_reassigned)
                                cluster_predicted_reassigned[slide] =clusters[cluster_number].predict(predd).ravel() 
                                #add new one at the end (what we've just predicted)
                                if horizon == False:
                                    cluster_data_X_reassigned[slide] = cluster_predicted_reassigned[slide].values
                                else:
                                    cluster_data_X_reassigned[slide] =(cluster_data_true_values_reassigned.iloc[:,slide]) 
                            # Save prediction
                            cluster_predicted_reassigned = pd.DataFrame(cluster_predicted_reassigned)
                            cluster_error[cluster_lowest_mae] = score_function(cluster_data_true_values_reassigned.values.ravel(),cluster_predicted_reassigned.values.ravel())
                            cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
        if (i==0):
            old_cluster_for_user = cluster_for_user.copy() # as well as save last assigned clusters                  
        print('Moved ', ii, ' users to another cluster \n')   
   # return old_clusters,old_cluster_for_user,cluster_parameters_for_gridsearch   

In [96]:
def final_prediction_after_clusterwise_reg_sklearn(
                           X,Y,regr,
    dataset,
    method,
    frequency,
    model,
                            horizon= True,
                           step = 'unstated',
                           clusters = None,
                           clusters_parameters = None,
                           new_clusters_for_users=None,
                           cluster_column ='supercluster',
                           scores_per_cluster = True,
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           score_function = (lambda x,y: math.sqrt(mean_squared_error(x,y))),
                           gridsearch_use = False,):
    """
    Performs final prediction and generates DataFrame for Alibaba Data
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    horizon : stated previously
    step : 'before_classifier' or 'after_classifier', for report generation
    clusters: dictionary with models for each cluster. If None, then refit
    clusters_parameters: dictionary with parameters for each cluster model
    new_clusters_for_users: dictionary with clusters for each user. 
    cluster_column : column with given clusters
    
    scores_per_cluster : if False the score will be given for the all dataset, not by cluster
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    score_function : smape/ pinball_loss/ mape/(lambda x,y: math.sqrt(mean_squared_error(x,y)))
    gridsearch_use : Run gridSeach (True) or use clusters_parameters (False)?
    regr : if not tmrf, then use sklearn api
    regr_gridsearch_params : if not tmrf, then use sklearn api for gridsearchCV
    ---------
    Return :
    result : DataFrame with columns
    cluster, user id, true target, prediction, rmse, model, method (all data, clustered), step (all data, before classifier, after classifier)
    """
    result = pd.DataFrame()
    train_targets = X.copy()
    test_targets = Y.copy()
    n_ahead = test_targets.shape[1]
    if (new_clusters_for_users is not None):
        # we assume that the order in DataFrame is the same as in clusterwise regression
        train_targets[cluster_column]=list(cluster_per_user.values()) 
    cluster_data_X = {}
    cluster_data_true_values = {}
    if (clusters_parameters == None or gridsearch_use==True):
        clusters_parameters = {}
    if (clusters == None or clusters_parameters ==None):
        clusters = {}
    for cluster_number in train_targets[cluster_column].unique():
        cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
        cluster_data_true_values[cluster_number] = test_targets.iloc[ cluster_data_X[cluster_number].index]
        cluster_data_true_values[cluster_number].index =   test_targets.iloc[ cluster_data_X[cluster_number].index].index
        predicted =pd.DataFrame()
        
        #if (cluster_number not in clusters or ( cluster_number not in  clusters_parameters) or gridsearch_use==True):
        clusters[cluster_number] = clone(regr)
            
        for slide in list(range(0,test_targets.shape[1])):
            #Train
            transformer = StandardScaler(with_mean=True, with_std=True)
            #print("X_train = ",cluster_data_X[cluster_number].iloc[:,:-1].columns)
            fitted_ = transformer.fit_transform(cluster_data_X[cluster_number].iloc[:,:-1]) # X - all col except last one

            clusters[cluster_number].fit(fitted_,cluster_data_X[cluster_number].iloc[:,-1:].values) # y - last column
            #Test
            #delete first col
            cluster_data_X[cluster_number] = cluster_data_X[cluster_number].drop(cluster_data_X[cluster_number].columns[0], axis=1)  
            #print("X_test = ",cluster_data_X[cluster_number].columns)
            fitted_ = transformer.fit_transform(cluster_data_X[cluster_number])
            predicted[slide] =clusters[cluster_number].predict(fitted_).ravel() 
            #add new one at the end (what we've just predicted)
            if (horizon == False):
                cluster_data_X[cluster_number][slide] = predicted[slide].values 
            else:
              #  print(cluster_data_true_values[cluster_number].iloc[:,slide])
                cluster_data_X[cluster_number][slide] =(cluster_data_true_values[cluster_number].iloc[:,slide]).values   # true values
        
        
        df_prediction = pd.DataFrame(predicted)
        df_prediction.columns  = cluster_data_true_values[cluster_number].columns 
        df_prediction.index = cluster_data_true_values[cluster_number].index
       # df_prediction.index =  train_targets.loc[train_targets[cluster_column] == cluster_number]["Unnamed: 1"]
        # Transform prediction for output
        #MODEL        
        
        df_prediction.insert(0,'Score',score_function(cluster_data_true_values[cluster_number].values.ravel(),df_prediction.values.ravel(),training_series=fitted_))
        df_prediction.insert(0,'model',model)
        #Dataset
        df_prediction.insert(0,'dataset',dataset)
        #METHOD
        df_prediction.insert(0,'method',method)
        #STEP
        df_prediction.insert(0,'frequency',frequency)
        if horizon == True:
            df_prediction.insert(0,'horizon',str(1)+'_step')
        else:
            df_prediction.insert(0,'horizon','full')
        # CLUSTER
        df_prediction.insert(0,'cluster_id',str(cluster_column)+'_'+str(cluster_number))
        #Wide to logn
        #print(pd.DataFrame(df_prediction))
        df_prediction = df_prediction.reset_index().melt(id_vars =['V1','cluster_id','model','horizon','dataset','method','frequency','Score']).rename(index = str, columns={'variable':'date','value':'predicted_value','V1':'id'})
        true_values=cluster_data_true_values[cluster_number].reset_index().melt(id_vars=['V1']).rename(index = str, columns={'variable':'date','value':'true_value','V1':'id'})
        result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))
    if scores_per_cluster == False:
           result.Score = score_function(result.true_value,result.predicted_value,
                             training_series=train_targets.drop(columns =columns_wo_TS).values.ravel())
    return result   

In [97]:
def clusterwise_regression(X,Y,
                           cluster_column ='supercluster',
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           gridsearch_limit = 1,
                           recalculate_assignment = 100,
                           score_function = (lambda x,y: math.sqrt(mean_squared_error(x,y))),
                          ):
    """
    Runs clusterwise regression
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    cluster_column : column with given clusters
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    gridsearch_limit : # of times to run GridSeach for each cluster (>=1)
    recalculate_assignment : # of assignments after which the clusters will be recalculated
    score_function : smape/ pinball_loss/ mape/(lambda x,y: math.sqrt(mean_squared_error(x,y)))
    regr : if not tmrf, then use sklearn api
    regr_gridsearch_params : if not tmrf, then use sklearn api for gridsearchCV
    ---------
    Return :
    clusters : Dictionary with trained models for each cluster
    old_cluster_for_user : Dictionary with optimal cluster for each user (after moving)
    cluster_parameters_for_gridsearch : Dictionary with optimal parameters for each cluster model
    weights
    """
        
        
    train_targets = X.copy()
    test_targets = Y.copy()  # test_Y 
    n_ahead = test_targets.shape[1]
    clusters = {}
    # save cluster info for clusterwise regression
    cluster_error = {}
    cluster_has_changed = {}
    cluster_data_X = {}
    counter_for_gridsearch = 0
    old_clusters ={}
    counter_iterations = 0 
    cluster_data_true_values = {}
    cluster_predicted = {}
    score_for_user = {}
    cluster_parameters_for_gridsearch = {}
    previous_total_error = math.inf
    limit = 50
    for i in list(range(0,limit+1)): # Some limit on iterations just in case
        # Assign user to clusters
        cluster_for_user = {}
        cluster_for_user  = train_targets[cluster_column].to_dict()
        for cluster_number in train_targets[cluster_column].unique():
            cluster_predicted[cluster_number] =pd.DataFrame()
            # Save training data for this cluster
            cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
            cluster_data_true_values[cluster_number] = test_targets.loc[ cluster_data_X[cluster_number].index]
            # Scaling (Not minmax because of it wasn't used in trmf github)
            transformer = StandardScaler(with_mean=True, with_std=True)
            cluster_data_X[cluster_number] = cluster_data_X[cluster_number].transpose()
           # print(cluster_number)
           # print(cluster_data_X[cluster_number])
            cluster_data_X[cluster_number] = transformer.fit_transform(cluster_data_X[cluster_number])
            #GridSearchCV for TRMF
            # If initial initialization or cluster has changed
            if (cluster_number not in cluster_has_changed) or (cluster_has_changed[cluster_number]== True):
                if (i<gridsearch_limit):
                    # add prev_cluster_paraeters
                    cluster_parameters_for_gridsearch[cluster_number] = gridsearch_trmf(cluster_data_X[cluster_number],cluster_data_true_values[cluster_number])
                clusters[cluster_number ]= TRMFRegressor(**cluster_parameters_for_gridsearch[cluster_number], eta_F=0., adj=None, C_B=0., fit_regression=False,
                                 fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                                 z_step_kind="tron")
                clusters[cluster_number].fit(cluster_data_X[cluster_number])
            fitted_ = np.dot(clusters[cluster_number].factors_,clusters[cluster_number].loadings_)+clusters[cluster_number].intercept_
            cluster_predicted[cluster_number] = transformer.inverse_transform( clusters[cluster_number].predict(fitted_,n_ahead))
            
            # Save prediction
            cluster_predicted[cluster_number] = pd.DataFrame(cluster_predicted[cluster_number]).transpose()
            
           # print(cluster_predicted[cluster_number].columns )
           # print( cluster_data_true_values[cluster_number].columns)
            cluster_predicted[cluster_number].columns  = cluster_data_true_values[cluster_number].columns 
            cluster_predicted[cluster_number].index = cluster_data_true_values[cluster_number].index
            #save cluster score
            cluster_error[cluster_number] =  score_function(cluster_data_true_values[cluster_number].values.ravel(),cluster_predicted[cluster_number].values.ravel())
        print("Error per cluster: ",cluster_error)
        # flatten true values
        all_true_values  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_data_true_values.values())  ]])   )
        # flatten prediction 
        all_predictions  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_predicted.values())  ]])   )
        total_error =  score_function(np.array(all_true_values),np.array(all_predictions))
        print("New error: ",round(total_error,4))
        # If new score is worse - load last cluster and run final prediction. 
        if (total_error > previous_total_error):
            if counter_iterations ==3:
               # train_targets[cluster_column] = list(old_cluster_for_user.values())
                print('Error has increased. Backtrack to previous iteration')
                return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch,i-counter_iterations
            counter_iterations+=1
        else:
            counter_iterations = 0 
            previous_total_error = total_error
            old_clusters = clusters.copy()
            old_cluster_for_user = cluster_for_user.copy()
            # Save all stuff
        if (i>=limit):# return prev iteration
            return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch,i
        # Calculate score per user
        for cluster_number in train_targets[cluster_column].unique():
            for index, row in cluster_data_true_values[cluster_number].iterrows():
                score_for_user[index] = score_function(row, cluster_predicted[cluster_number].loc[index,:])
        # Assume that we haven't change any cluster
        for x in cluster_has_changed:
            cluster_has_changed[x]=False        
        cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
       # old_clusters = {}
        #old_cluster_for_user = {}
        ii =0
        reassignment_count = 0
        if (i==0):
            old_clusters = clusters.copy()
        #Take observation x from cluster a, where error_x > error_a
        for cluster_number in train_targets[cluster_column].unique():
            # All ids in this cluster are already in cluster with lowest possible mae
            if (cluster_number == cluster_lowest_mae):
                continue
            for index, row in cluster_data_true_values[cluster_number].iterrows():    
            # User is in current cluster -> we can reassign him
                if (cluster_number  == cluster_for_user[index]):
                    if (score_for_user[index] > cluster_error[cluster_number]):
                        ii+=1#counter
                        # Have to refit changed clusters             
                        cluster_has_changed[cluster_number] = True
                        cluster_has_changed[cluster_lowest_mae] = True
                        # Move observation x to the cluster with the lowest training erro
                        train_targets.loc[index,cluster_column] = cluster_lowest_mae
                        reassignment_count+=1
                        if reassignment_count==recalculate_assignment:
                            reassignment_count=0
                            # RECALCULATION - only best performing cluster (because)- TODO:save snapshot of prev cluster for backtracking
                            cluster_data_X_reassigned =  train_targets.loc[train_targets[cluster_column] == cluster_lowest_mae].drop(columns =columns_wo_TS)
                            
                            # print(  train_targets.loc[train_targets.cluster==2].drop(columns = ['cluster','V1']).shape)
                            cluster_data_true_values_reassigned= test_targets.loc[ cluster_data_X_reassigned.index]
                            cluster_data_X_reassigned =  transformer.fit_transform(cluster_data_X_reassigned.transpose())
                            clusters_reassignemnt = clone(clusters[cluster_lowest_mae])
                            clusters_reassignemnt.fit(cluster_data_X_reassigned)
                            predd =  transformer.inverse_transform(clusters_reassignemnt.predict(cluster_data_X_reassigned ,n_ahead=n_ahead))
                            cluster_predicted_reassigned = predd
                            # Save prediction
                            cluster_predicted_reassigned = pd.DataFrame(cluster_predicted_reassigned).transpose()
                            cluster_error[cluster_lowest_mae] = score_function(cluster_data_true_values_reassigned.values.ravel(),cluster_predicted_reassigned.values.ravel())
                            cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
        if (i==0):
            old_cluster_for_user = cluster_for_user.copy() # as well as save last assigned clusters                  
        print('Moved ', ii, ' users to another cluster \n')   
   # return old_clusters,old_cluster_for_user,cluster_parameters_for_gridsearch   

In [98]:
def final_prediction_after_clusterwise_reg(
    X,Y,
        dataset,
    method,
    frequency,
    model,           
                           step = 'unstated',
                           clusters = None,
                           clusters_parameters = None,
                           new_clusters_for_users=None,
                           cluster_column ='supercluster',
                           scores_per_cluster = True,
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           score_function = (lambda x,y: math.sqrt(mean_squared_error(x,y))),
                           gridsearch_use = False):
    """
    Performs final prediction and generates DataFrame for Alibaba Data
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    step : 'before_classifier' or 'after_classifier', for report generation
    clusters: dictionary with models for each cluster. If None, then refit
    clusters_parameters: dictionary with parameters for each cluster model
    new_clusters_for_users: dictionary with clusters for each user. 
    cluster_column : column with given clusters
    
    scores_per_cluster : if False the score will be given for the all dataset, not by cluster
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    score_function : smape/ pinball_loss/ mape/(lambda x,y: math.sqrt(mean_squared_error(x,y)))
    gridsearch_use : Run gridSeach (True) or use clusters_parameters (False)?
    regr : if not tmrf, then use sklearn api
    regr_gridsearch_params : if not tmrf, then use sklearn api for gridsearchCV
    ---------
    Return :
    result : DataFrame with columns
    cluster, user id, true target, prediction, rmse, model, method (all data, clustered), step (all data, before classifier, after classifier)
    """
    result = pd.DataFrame()
    train_targets = X.copy()
    test_targets = Y.copy()
    n_ahead = test_targets.shape[1]
    if (new_clusters_for_users is not None):
        # we assume that the order in DataFrame is the same as in clusterwise regression
        train_targets[cluster_column]=list(cluster_per_user.values()) 
    cluster_data_X = {}
    cluster_data_true_values = {}
    if (clusters_parameters == None or gridsearch_use==True):
        clusters_parameters = {}
    if (clusters == None or clusters_parameters ==None):
        clusters = {}
    for cluster_number in train_targets[cluster_column].unique():
        cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
        cluster_data_true_values[cluster_number] = test_targets.iloc[ cluster_data_X[cluster_number].index]
        cluster_data_true_values[cluster_number].index =   test_targets.iloc[ cluster_data_X[cluster_number].index].index
        # Scaling (Not minmax because of it wasn't used in trmf github)
        transformer = StandardScaler(with_mean=True, with_std=True)
        
        cluster_data_X[cluster_number] = cluster_data_X[cluster_number].transpose()
        cluster_data_X[cluster_number] = transformer.fit_transform(cluster_data_X[cluster_number])
        if (cluster_number not in clusters_parameters or gridsearch_use==True):
            clusters_parameters[cluster_number] = gridsearch_trmf(cluster_data_X[cluster_number],cluster_data_true_values[cluster_number])
                    
        if (cluster_number not in clusters or ( cluster_number not in  clusters_parameters) or gridsearch_use==True):
            clusters[cluster_number ]= TRMFRegressor(**clusters_parameters[cluster_number], eta_F=0., adj=None, C_B=0., fit_regression=False,
                             fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                             z_step_kind="tron")
                # If you feed clusters from clusterwise, you don't need to refit them
            clusters[cluster_number ].fit(cluster_data_X[cluster_number])
        fitted_ = np.dot(clusters[cluster_number].factors_,clusters[cluster_number].loadings_)+clusters[cluster_number].intercept_
        predicted = transformer.inverse_transform( clusters[cluster_number].predict(fitted_,n_ahead))

            
        df_prediction = pd.DataFrame(predicted).transpose()
        df_prediction.columns  = cluster_data_true_values[cluster_number].columns 
        df_prediction.index = cluster_data_true_values[cluster_number].index # here we need to set old cluster
    
        # Transform prediction for output
        #MODEL
        df_prediction.insert(0,'Score',score_function(cluster_data_true_values[cluster_number].values.ravel(),df_prediction.values.ravel(),training_series=fitted_))
        df_prediction.insert(0,'model',model)
        #Dataset
        df_prediction.insert(0,'dataset',dataset)
        #METHOD
        df_prediction.insert(0,'method',method)
        #STEP
        df_prediction.insert(0,'frequency',frequency)
        df_prediction.insert(0,'horizon','full')
        df_prediction.insert(0,'cluster_id',str(cluster_column)+'_'+str(cluster_number))
        #Wide to logn
        df_prediction = df_prediction.reset_index().melt(id_vars =['V1','cluster_id','model','horizon','dataset','method','frequency','Score']).rename(index = str, columns={'variable':'date','value':'predicted_value','V1':'id'})
        true_values=cluster_data_true_values[cluster_number].reset_index().melt(id_vars=['V1']).rename(index = str, columns={'variable':'date','value':'true_value','V1':'id'})
        result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))
    if scores_per_cluster == False:
           result.Score = score_function(result.true_value,result.predicted_value,
                             training_series=train_targets.drop(columns =columns_wo_TS).values.ravel())
    return result   

In [99]:
# TODO: Check if it can work with Gridsearch CV(It should but, you can just pass None)
class RegressorEnsemble(BaseEstimator):
    def __init__(self, rgr, n_estimators=10):
        self.rgr = rgr
        self.n_estimators = n_estimators
        self.rgrs = []
        for i in range(self.n_estimators):
            self.rgrs.append(clone(self.rgr))

    def fit(self, X, y,
              seed=None,):
        if seed is not None:
            np.random.seed(seed)
        #sd = np.random()
        #print(sd)
        for i in range(self.n_estimators):
            self.rgrs[i].fit(X, y)

    def predict(self, X):
        ans = self.rgrs[0].predict(X)
        for i in range(1,self.n_estimators):
            ans += self.rgrs[i].predict(X)
        return ans / len(self.rgrs)

In [100]:
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,max_depth=3,
                                                          random_state=0))
regr_multirf = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror',
                                                     n_estimators=50)) #tree_method='gpu_hist'))

# Reporting

In [101]:
res = pd.DataFrame()

In [102]:
#train_yearly.shape  - okay shape (1 min)

In [103]:
train_weekly.shape

(359, 2602)

In [104]:
test_weekly.shape

(359, 14)

In [105]:
delete_column_limit = 2575 # 1 - without deletion

## SKLEARN 

In [109]:
%%time
freq = 'weekly'
# FOR TRMF
for col in ['random initialization2','random initialization1', 'GMM_Voting','GMM_Pair']:
    X = train_weekly.drop(columns = ['V'+str(x) for x in list(range(2,delete_column_limit))])
    Y  = test_weekly
    reggr = XGBRegressor(n_estimators=100)
    #reggr = Ridge()
    modl = 'XGBoost'
   
    clusters, cluster_per_user,parameters,reassignment_iterations= clusterwise_regression_sklearn(X,#.drop(columns =['V'+str(x) for x in list(range(2,817))]),
                                                                        Y.drop(columns = 'V1'),
                                                                        cluster_column=col,
                                                                            columns_wo_TS=['random initialization1','V1','random initialization2', 'GMM_Voting','GMM_Pair'],
                                                                            score_function=pinball_loss,
                                                                           recalculate_assignment=5,
                                                                          horizon = False,
                                                                           regr = reggr,
                                                                          regr_gridsearch_params = None)
    for scorer in [MASE,smape]:
        dff_sklearn = final_prediction_after_clusterwise_reg_sklearn(X,
                                            Y.set_index( 'V1'),
                                            regr = reggr,
                                            clusters = clusters,
                                            clusters_parameters = parameters,
                                            new_clusters_for_users = cluster_per_user,
                                            gridsearch_use = None,
                                            cluster_column=col,
                                            scores_per_cluster = False , #important !False,
                                            score_function=scorer,#smape,#pinball_loss,#  (lambda x,y: math.sqrt(mean_squared_error(x,y)))
                                            columns_wo_TS=['random initialization1','V1','random initialization2', 'GMM_Voting','GMM_Pair'],
                                            horizon = False,

                                            dataset = 'M4',
                                            model = modl,
                                            method = 'clusterwise',
                                            frequency = freq)

        dff_sklearn['clusterwise_loss'] = 'PINBALL'
        if scorer==MASE:
            dff_sklearn['final_loss'] = 'MASE'
        if scorer==smape:
            dff_sklearn['final_loss'] = 'SMAPE'
        dff_sklearn['reassignments_iterations'] = reassignment_iterations
        res = res.append(dff_sklearn)
        print(res.shape)

Error per cluster:  {3: 90.70531176539522, 4: 127.23504885304376, 5: 145.65682978846036, 0: 182.86138600421285, 6: 297.09073385761883, 1: 173.55534017533316, 2: 167.51755607731442}
New error:  165.6743
Moved  81  users to another cluster 

Error per cluster:  {3: 162.80927445479358, 4: 102.8289667293589, 5: 203.43471020070496, 0: 205.20499727347018, 6: 122.46548880951458, 1: 216.72305726528822, 2: 142.0200746665872}
New error:  166.1038
Moved  51  users to another cluster 

Error per cluster:  {3: 87.26797709885535, 4: 116.73736163980654, 5: 170.23448256152068, 0: 227.10013562741818, 6: 185.27036710220727, 1: 257.4230385547614, 2: 216.80056837078155}
New error:  177.5924
Moved  35  users to another cluster 

Error per cluster:  {3: 82.94585730564334, 4: 112.28123278260452, 5: 118.4969365645506, 0: 116.94306274790475, 6: 192.97470146037313, 1: 274.9622920226908, 2: 239.25294496252613}
New error:  166.505
Moved  56  users to another cluster 

Error per cluster:  {3: 131.8197714576499, 4:

Moved  42  users to another cluster 

Error per cluster:  {7: 121.32479285410426, 1: 176.1006556394241, 2: 90.80708245813209, 0: 120.27189656850679, 5: 86.04914102564103, 6: 414.5708172527377, 3: 179.53082952411356, 4: 263.4144954661834}
New error:  159.6148
Moved  50  users to another cluster 

Error per cluster:  {7: 326.3993714417645, 1: 174.37214151580795, 2: 148.71923976928838, 0: 96.7286363379038, 5: 110.98405610069861, 6: 41.50636017628205, 3: 128.7814042005252, 4: 342.81244914944597}
New error:  182.2487
Moved  78  users to another cluster 

Error per cluster:  {7: 75.08630842659922, 1: 197.11569664173717, 2: 87.63188038683772, 0: 90.70933981714535, 5: 163.9028231609765, 6: 558.3244467707154, 3: 279.0608215774704, 4: 80.20956637427562}
New error:  154.8893
Moved  67  users to another cluster 

Error per cluster:  {7: 149.00578445628005, 1: 397.95890492930846, 2: 83.5845962959427, 0: 91.24945448995774, 5: 180.44801513397047, 6: 113.23508789062498, 3: 141.288436741575, 4: 142.025

## TRMF

In [33]:
%%time
freq = 'weekly'
# FOR TRMF
for col in ['random initialization2','random initialization1', 'GMM_Voting','GMM_Pair']:
    X = train_weekly.drop(columns = ['V'+str(x) for x in list(range(2,delete_column_limit))])
    Y  = test_weekly
    clusters, cluster_per_user,parameters,reassignment_iterations = clusterwise_regression(X,#.drop(columns =['V'+str(x) for x in list(range(2,817))]),
                                                                    Y.drop(columns = 'V1'),
                                                                    cluster_column=col,
                                                                        columns_wo_TS=['random initialization1','V1','random initialization2', 'GMM_Voting','GMM_Pair'],
                                                                        score_function=pinball_loss,
                                                                       recalculate_assignment=(10000))
    
    for scorer in [MASE,smape]:
        dff_trmf = final_prediction_after_clusterwise_reg(X,
                                        Y.set_index( 'V1'),
                                        step ='before_classifier',
                                        clusters = clusters,
                                        clusters_parameters = parameters,
                                        new_clusters_for_users = cluster_per_user,
                                        gridsearch_use = None,
                                        cluster_column=col,
                                        scores_per_cluster = False , #important !False,
                                        score_function=scorer,#smape,#pinball_loss,#  (lambda x,y: math.sqrt(mean_squared_error(x,y)))
                                        columns_wo_TS=['random initialization1','V1','random initialization2', 'GMM_Voting','GMM_Pair'],

                                        dataset = 'M4',
                                        model = 'TRMF',
                                        method = 'clusterwise',
                                        frequency = freq)
        
        dff_trmf['clusterwise_loss'] = 'PINBALL'
        if scorer==MASE:
            dff_trmf['final_loss'] = 'MASE'
        if scorer==smape:
            dff_trmf['final_loss'] = 'SMAPE'
        dff_trmf['reassignments_iterations'] = reassignment_iterations
        res = res.append(dff_trmf)
        print(res.shape)

Error per cluster:  {1: 249.9186070443019, 0: 234.33858911635022, 6: 226.44025287800685, 2: 176.87775442196798, 5: 255.95235176744336, 4: 263.4964034720618, 3: 240.47549641570134}
New error:  236.3781
Moved  83  users to another cluster 

Error per cluster:  {1: 39.13692561687852, 0: 51.12693622796588, 6: 63.14988014232318, 2: 585.104064804056, 5: 44.31067969499242, 4: 28.94812109321283, 3: 54.09085959241898}
New error:  236.2512
Moved  102  users to another cluster 

Error per cluster:  {1: 3.3475736806207608, 0: 6.284657290768002, 6: 15.054617641600156, 2: 242.93261142060527, 5: 8.3046618816738, 4: 463.23418705645105, 3: 9.570384161159124}
New error:  236.2626
Moved  121  users to another cluster 

Error per cluster:  {1: 482.34754824573434, 0: 0.0, 6: 0.0, 2: 54.76676118927646, 5: 0.9038914141862682, 4: 103.65249267764979, 3: 0.028648772847189474}
New error:  236.2932
Moved  116  users to another cluster 

Error per cluster:  {1: 135.10267458450602, 0: 512.6029803670608, 6: 0.0, 2: 

## Saving file

In [110]:
import pandas as pd
import time


def write_df(df: pd.DataFrame,
             path: str,
             project: str,
             step: str,
             format: str = 'csv') -> None:


    title = path + time.strftime("%Y%m%d_%H%M") + '_' + project + '__' + step + '.' + format

    if format == 'csv':
        df.to_csv(title)

    if format == 'h5':
        df.to_hdf(title, step, table=True, mode='a')

In [111]:
write_df(res, export,'weekly_xgb_final2',str(train_weekly.shape[0])+'_'+str(delete_column_limit)+'+')